In [1]:
"""
============================================================
Parameter estimation using grid search with cross-validation
============================================================

This examples shows how a classifier is optimized by cross-validation,
which is done using the :class:`sklearn.grid_search.GridSearchCV` object
on a development set that comprises only half of the available labeled data.

The performance of the selected hyper-parameters and trained model is
then measured on a dedicated evaluation set that was not used during
the model selection step.

More details on tools available for model selection can be found in the
sections on :ref:`cross_validation` and :ref:`grid_search`.

"""

from __future__ import print_function

from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from random import randrange

print(__doc__)



Parameter estimation using grid search with cross-validation

This examples shows how a classifier is optimized by cross-validation,
which is done using the :class:`sklearn.grid_search.GridSearchCV` object
on a development set that comprises only half of the available labeled data.

The performance of the selected hyper-parameters and trained model is
then measured on a dedicated evaluation set that was not used during
the model selection step.

More details on tools available for model selection can be found in the
sections on :ref:`cross_validation` and :ref:`grid_search`.




In [2]:

# Loading the Digits dataset
digits = datasets.load_digits()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target


In [3]:
def listadd(l1, l2):
    return ([l1[i] + l2[i] for i in range(min(len(l1), len(l2)))])

In [4]:
def listmult(l1, factor):
    return ([l1[i] *factor for i in range(len(l1))])


In [5]:
#put noise in featureset
shadowFactor=0 #.3
noiseRange=0

for record in range(n_samples):
    #add shadows
    X[record]=listadd(X[record], listmult(X[randrange(0, n_samples-1)], shadowFactor))
    
    #add random noise
    for datapoint in range(len(X[0])):
        if noiseRange != 0:
            X[record][datapoint]+=randrange(-1*noiseRange,noiseRange)
        
        #quantize the results and put in a limited range
        X[record][datapoint]=int(max(0, min(15, X[record][datapoint])))


In [6]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)


In [7]:
# Set the parameters by cross-validation
tuned_parameters = [#{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     #'C': [1, 10, 100, 1000, 10000]},
                    #{'kernel': ['linear'], 'C': [1, 10, 100, 1000, 10000]},
                   #{'kernel':['poly'], 'C': [1, 10, 100, 1000, 10000], 'degree':[1,2,3,4,5], 'coef0':[0,-.1,.1]},
                   {'kernel':['sigmoid'], 'C': [1, 10, 100, 1000], 'gamma': [5e-4, 1e-4, 5e-5], 'coef0':[0,-.1,.1]}
                   ]

scores = ['precision', 'recall']
print(tuned_parameters)


[{'coef0': [0, -0.1, 0.1], 'C': [1, 10, 100, 1000], 'gamma': [0.0005, 0.0001, 5e-05], 'kernel': ['sigmoid']}]


In [8]:
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5,
                       scoring='%s_weighted' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'kernel': 'sigmoid', 'C': 1000, 'gamma': 5e-05, 'coef0': -0.1}

Grid scores on development set:

0.916 (+/-0.014) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.0005, 'coef0': 0}
0.943 (+/-0.025) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.0001, 'coef0': 0}
0.921 (+/-0.038) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 5e-05, 'coef0': 0}
0.915 (+/-0.019) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.0005, 'coef0': -0.1}
0.943 (+/-0.025) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.0001, 'coef0': -0.1}
0.922 (+/-0.039) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 5e-05, 'coef0': -0.1}
0.911 (+/-0.021) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.0005, 'coef0': 0.1}
0.940 (+/-0.029) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.0001, 'coef0': 0.1}
0.922 (+/-0.037) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 5e-05, 'coef0': 0.1}
0.824 (+/-0.050) for {'kernel': 'sigmoid', 'C': 10, 'gamma': 0.0005, 'coef0': 0}
0